In [1]:
import polars as pl

import polars_ts as pts  # noqa

# Create sample dataframe with columns `unique_id`, `ds`, and `y`.
df = (
    pl.scan_parquet("https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet")
    .filter(pl.col("unique_id").is_in(["H1", "H2", "H3"]))
    .group_by("unique_id").agg(pl.col("y").head(5))
    .explode("y")
    .with_columns((pl.col("y") - pl.mean("y")) / pl.std("y"))
    .collect()
)
df

unique_id,y
str,f64
"""H1""",-1.161573
"""H1""",-1.180977
"""H1""",-1.180977
"""H1""",-1.208551
"""H1""",-1.257571
…,…
"""H3""",0.087419
"""H3""",0.064952
"""H3""",0.157886


In [2]:
pts.compute_pairwise_dtw(df, df)

id_1,id_2,dtw
str,str,f32
"""H2""","""H1""",11.726028
"""H2""","""H2""",0.0
"""H2""","""H3""",5.483109
"""H1""","""H1""",0.0
"""H1""","""H2""",11.726028
"""H1""","""H3""",6.24292
"""H3""","""H1""",6.24292
"""H3""","""H2""",5.483109
"""H3""","""H3""",0.0
